In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
from ml_experiments.analyze import get_df_runs_from_mlflow_sql, get_missing_entries
from functools import partial

# Save Results

## Load mlflow runs

In [2]:
db_port = 5101
db_name = 'cohirf'
w = 'clust9'
# url = f'postgresql://{w}.ceremade.dauphine.lan:{db_port}/{db_name}'
url = f'postgresql://beluccib@localhost:{db_port}/{db_name}'
engine = create_engine(url)
query = 'SELECT experiments.name from experiments'
experiment_names = pd.read_sql(query, engine)['name'].tolist()
# results_dir = Path('~/tab_benchmark/results')
# os.makedirs(results_dir, exist_ok=True)

In [3]:
experiment_names

['Default',
 'classif_huge_cohirf',
 'classif_huge_kmeans',
 'classif_clusters_kmeans',
 'classif_clusters_cohirf',
 'classif_huge_cohirf_sampling',
 'classif_huge_cohirf_minibatch',
 'time_hc_CompleteAgglomerativeClustering_samples',
 'time_hc_WardAgglomerativeClustering_samples',
 'time_hc_OPTICS_samples',
 'time_hc_CompleteAgglomerativeClustering_features',
 'time_hc_Proclus_features',
 'time_hc_WardAgglomerativeClustering_features',
 'time_hc_KMeans_features',
 'time_hc_KMeansProj_samples',
 'time_hc_Clique_samples',
 'time_hc_DBSCAN_features',
 'time_hc_KMeans_samples',
 'time_hc_DBSCAN_samples',
 'time_hc_AffinityPropagation_samples',
 'time_hc_RecursiveClustering_features',
 'time_hc_SpectralSubspaceRandomization_samples',
 'time_hc_KMeansProj_features',
 'time_hc_MeanShift_features',
 'time_hc_HDBSCAN_features',
 'time_hc_OPTICS_features',
 'time_hc_Clique_features',
 'time_hc_IRFLLRR_features',
 'time_hc_RecursiveClustering_samples',
 'time_hc_MeanShift_samples',
 'time_hc_Spe

In [4]:
experiments_names = [exp for exp in experiment_names if exp.find('openml')!= -1 and exp.find('openml_task')==-1]

In [5]:
params_columns = [
    'model_nickname',
    'n_samples',
    'n_features',
    'n_classes',
    'seed_model',
    'dataset_id',
    'dataset_name',
    'model_params/representative_method',
    'model_params/n_samples_representative',
    'model_params/kernel_kmeans',
    'direction',
    'hpo_metric',
    'standardize'
]

In [6]:
latest_metrics_columns = [
    'fit_model_return_elapsed_time',
    'max_memory_used',
    'n_clusters_',
    'rand_score',
    'adjusted_rand',
    'mutual_info',
    'adjusted_mutual_info',
    'normalized_mutual_info',
    'homogeneity',
    'completeness',
    'v_measure',
    'silhouette',
    'calinski_harabasz_score',
    'davies_bouldin_score',
    'inertia_score',
    'best_n_clusters_',
    'best_rand_score',
    'best_adjusted_rand',
    'best_mutual_info',
    'best_adjusted_mutual_info',
    'best_normalized_mutual_info',
    'best_homogeneity_completeness_v_measure',
    'best_silhouette',
    'best_calinski_harabasz_score',
    'best_davies_bouldin_score',
    'best_inertia_score',
    'best_homogeneity',
    'best_completeness',
    'best_v_measure',
]

In [7]:
tags_columns = [
    'raised_exception',
    'EXCEPTION',
    'parent_run_id',
    'best_child_run_id',
]

In [8]:
# parameters of each model, they are saved as tags for the parent run
parameters = {
    'RecursiveClustering': ['components_size', 'repetitions', 'kmeans_n_clusters'],
    'KMeans': ['n_clusters'],
    'HDBSCAN': ['min_cluster_size'],
    'DBSCAN': ['eps', 'min_samples'],
    'AffinityPropagation': ['damping'],
    'OPTICS': ['min_samples'],
    'SpectralSubspaceRandomization': ['n_similarities', 'sampling_ratio', 'sc_n_clusters'],
    'WardAgglomerativeClustering': ['n_clusters'],
    'Clique': ['n_partitions', 'density_threshold'],
    'IRFLLRR': ['sc_n_clusters', 'p', 'c', 'lambda_', 'alpha'],
    'KMeansProj': ['n_clusters'],
    'Proclus': ['n_clusters', 'avg_dims'],
    'SpectralClustering': ['n_clusters'],
    'SingleAgglomerativeClustering': ['n_clusters'],
    'CompleteAgglomerativeClustering': ['n_clusters'],
    'AverageAgglomerativeClustering': ['n_clusters'],
    'MeanShift': ['min_bin_freq'],
}
all_model_parameters = [p for model, params in parameters.items() for p in params]
all_model_parameters = list(set(all_model_parameters))
for param in all_model_parameters:
    tags_columns.append(param)

In [9]:
runs_columns = ['run_uuid', 'status', 'start_time', 'end_time']
experiments_columns = []
other_table = 'params'
other_table_keys = params_columns
df_params = get_df_runs_from_mlflow_sql(engine, runs_columns=runs_columns, experiments_columns=experiments_columns, experiments_names=experiments_names, other_table=other_table, other_table_keys=other_table_keys)
df_latest_metrics = get_df_runs_from_mlflow_sql(engine, runs_columns=['run_uuid'], experiments_columns=experiments_columns, experiments_names=experiments_names, other_table='latest_metrics', other_table_keys=latest_metrics_columns)
df_tags = get_df_runs_from_mlflow_sql(engine, runs_columns=['run_uuid'], experiments_columns=experiments_columns, experiments_names=experiments_names, other_table='tags', other_table_keys=tags_columns)

In [10]:
df_runs_raw = df_params.join(df_latest_metrics)
df_runs_raw = df_runs_raw.join(df_tags)

In [11]:
df_runs_raw_parents = df_runs_raw.copy()
df_runs_raw_parents = df_runs_raw_parents.loc[df_runs_raw_parents['parent_run_id'].isna()]

In [12]:
# get all metrics beside best or last
metrics_columns = [
    'adjusted_rand',
    'silhouette',
    'calinski_harabasz_score',
    'davies_bouldin_score',
    'inertia_score',
]
metrics_columns_query = [f"'{m}'" for m in metrics_columns]
metrics_columns_query = ', '.join(metrics_columns_query)
experiments_names_query = [f"'{name}'" for name in experiments_names]
experiments_names_query = ', '.join(experiments_names_query)
query = f"""
SELECT
	runs.run_uuid,
	metrics."key",
	metrics.value,
	metrics.step 
FROM
	runs
LEFT JOIN metrics ON
	metrics.run_uuid = runs.run_uuid
LEFT JOIN experiments ON 
	experiments.experiment_id = runs.experiment_id  
WHERE 
	metrics."key" IN ({metrics_columns_query})
	AND experiments."name" IN ({experiments_names_query})
"""
df_metrics = pd.read_sql(query, engine)
df_metrics = df_metrics.pivot_table(index=['run_uuid', 'step'], columns='key', values='value')
df_metrics

,key,adjusted_rand,calinski_harabasz_score,davies_bouldin_score,inertia_score,silhouette
run_uuid,step,,,,,
000bd31bf02444d592585bc542d4df5b,0,-0.042740,2.347953,0.665062,1.182605e+05,0.161783
002720a0dd1e499893174c161e95410f,0,0.095148,2.432060,1.074172,2.854436e+04,0.048543
002b40f8feac40b68e0b8045bd4d1597,0,0.000000,-1.000000,1000.000000,3.605180e+03,-1.000000
002c80864618429395e88192cef8533a,0,0.142890,2.301928,0.915314,9.033444e+04,0.077393
00314537240c486a98dbf78117a3c41c,0,0.300603,3.330224,2.160309,7.000641e+04,0.055164
...,...,...,...,...,...,...
ffa26d4096a74842a45417159f50f00a,0,0.016404,2.893411,0.575626,1.584268e+05,0.142035
ffaedace4f804208b4e621dd0c827210,0,0.159510,3.898134,1.224359,2.901377e+04,0.097607
ffc4640be38d4423a01514a3bdad668e,0,0.084148,2.373405,1.420516,4.814853e+04,0.031826


In [13]:
df_runs_raw_parents


key,status,start_time,end_time,dataset_id,dataset_name,direction,hpo_metric,model_nickname,n_classes,n_features,...,min_samples,n_clusters,n_partitions,n_similarities,p,parent_run_id,raised_exception,repetitions,sampling_ratio,sc_n_clusters
run_uuid,,,,,,,,,,,,,,,,,,,,,
000977a3bdd6401599eae1684ec59d0d,FAILED,1742922415890,1.742930e+12,46782,binary_alpha_digits,maximize,adjusted_rand,DBSCAN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
0152dc997af245d4baf79d0192b306ae,FAILED,1742922366888,1.742930e+12,46775,armstrong-2002-v1,maximize,adjusted_rand,KMeansProj,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
017f4aad072141019af28721a3408745,FAILED,1742922369405,1.742930e+12,46783,coil-20,maximize,adjusted_rand,SpectralClustering,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
01adf00b439041338d8fc130fb8ebbba,FINISHED,1743266411846,1.743285e+12,554,mnist_784,maximize,adjusted_rand,CompleteAgglomerativeClustering,NaN,NaN,...,NaN,13,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN
01b919d95d4c4aedada499d3cd3cc53f,FAILED,1742922368297,1.742930e+12,46774,alizadeh-2000-v3,maximize,adjusted_rand,CompleteAgglomerativeClustering,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fd7ce3adee7345ddb805b04404918f09,FINISHED,1742938538892,1.742939e+12,46778,chowdary-2006,maximize,adjusted_rand,MeanShift,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN
fdd8f9917a814c44998f5aba77f1c49e,RUNNING,1742997251963,NaN,46781,NaN,maximize,adjusted_rand,Clique,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fe990c4d8d9b484998e72e992aab22ee,FINISHED,1742938550295,1.742939e+12,46782,binary_alpha_digits,maximize,adjusted_rand,KMeans,NaN,NaN,...,NaN,26,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN


# Separate our method with two types of representative_method

In [14]:
df_runs_parents = df_runs_raw_parents.copy()

# # fill with default values
# df_runs_parents['model_params/representative_method'] = df_runs_parents['model_params/representative_method'].fillna('closest_overall')
# df_runs_parents['model_params/n_samples_representative'] = 'None' #df_runs_parents['model_params/n_samples_representative'].fillna('None')
# df_runs_parents['model_params/kernel_kmeans'] = df_runs_parents['model_params/kernel_kmeans'].fillna('False')
#
# # fix closest_overall_1000 with new params
# df_runs_parents.loc[(df_runs_parents['model_nickname']=='RecursiveClustering') & (df_runs_parents['model_params/representative_method']=='closest_overall_1000'), 'model_params/n_samples_representative'] = '1000'
# df_runs_parents.loc[(df_runs_parents['model_nickname']=='RecursiveClustering') & (df_runs_parents['model_params/representative_method']=='closest_overall_1000'), 'model_params/representative_method'] = 'closest_overall'
#
# df_runs_parents.loc[df_runs_parents['model_nickname']=='RecursiveClustering', 'model_nickname'] = df_runs_parents.loc[df_runs_parents['model_nickname']=='RecursiveClustering', 'model_nickname'] + '_' + df_runs_parents.loc[df_runs_parents['model_nickname']=='RecursiveClustering', 'model_params/representative_method'] + '_' + df_runs_parents.loc[df_runs_parents['model_nickname']=='RecursiveClustering', 'model_params/n_samples_representative'] + '_' + df_runs_parents.loc[df_runs_parents['model_nickname']=='RecursiveClustering', 'model_params/kernel_kmeans']

# df_runs_parents.loc[(df_runs_parents['model_nickname'] == 'RecursiveClustering') & (df_runs_parents['model_params/representative_method'].isna()), 'model_params/representative_method'] = 'closest_overall'
# df_runs_parents.loc[(df_runs_parents['model_nickname'] == 'RecursiveClustering') & (df_runs_parents['model_params/representative_method'] == 'rbf'), 'model_nickname'] = df_runs_parents.loc[(df_runs_parents['model_nickname'] == 'RecursiveClustering') & (df_runs_parents['model_params/representative_method'] == 'rbf'), 'model_nickname'] + '_rbf'
# df_runs_parents.loc[(df_runs_parents['model_nickname'] == 'RecursiveClustering') & (df_runs_parents['model_params/representative_method'] == 'closest_overall_1000'), 'model_nickname'] = df_runs_parents.loc[(df_runs_parents['model_nickname'] == 'RecursiveClustering') & (df_runs_parents['model_params/representative_method'] == 'closest_overall_1000'), 'model_nickname'] + '_aproximated'


## Delete duplicate runs (if any) and complete some models that cannot run with some datasets

In [15]:
non_duplicate_columns = [
    'model_nickname',
    'dataset_id',
    # 'model_params/normalization',
    'direction',
    'hpo_metric',
    'standardize',
]
df_runs_parents = df_runs_parents.dropna(axis=0, how='all', subset=['best_adjusted_rand']).copy()
df_runs_parents = df_runs_parents.loc[(~df_runs_parents.duplicated(non_duplicate_columns))]

 # Missing

In [16]:
model_nickname = df_runs_parents['model_nickname'].unique().tolist()
model_nickname.sort()
model_nickname

['AffinityPropagation',
 'AverageAgglomerativeClustering',
 'Clique',
 'CompleteAgglomerativeClustering',
 'DBSCAN',
 'HDBSCAN',
 'IRFLLRR',
 'KMeans',
 'KMeansProj',
 'MeanShift',
 'OPTICS',
 'Proclus',
 'RecursiveClustering',
 'RecursiveClustering_full',
 'SingleAgglomerativeClustering',
 'SpectralClustering',
 'SpectralSubspaceRandomization',
 'WardAgglomerativeClustering']

In [17]:
non_duplicate_columns = [
    'model_nickname',
    'dataset_id',
    'direction',
    'hpo_metric',
    'standardize',
]

In [18]:
model_nickname = df_runs_parents['model_nickname'].unique().tolist()
model_nickname.sort()
model_nickname

['AffinityPropagation',
 'AverageAgglomerativeClustering',
 'Clique',
 'CompleteAgglomerativeClustering',
 'DBSCAN',
 'HDBSCAN',
 'IRFLLRR',
 'KMeans',
 'KMeansProj',
 'MeanShift',
 'OPTICS',
 'Proclus',
 'RecursiveClustering',
 'RecursiveClustering_full',
 'SingleAgglomerativeClustering',
 'SpectralClustering',
 'SpectralSubspaceRandomization',
 'WardAgglomerativeClustering']

In [19]:
# dataset_id = datasets['dataset_id'].tolist()
dataset_id = [46773,46774,46775,46776,46777,46778,46779,46780,46781,46782,46783,]
# dataset_id = [554,40996]
dataset_id = [str(d) for d in dataset_id]
direction = ['maximize']
metric = ['adjusted_rand']
standardize = ['True']
model_nickname = [
    'AffinityPropagation',
    'AverageAgglomerativeClustering',
    'Clique',
    'CompleteAgglomerativeClustering',
    'DBSCAN',
    'HDBSCAN',
    'IRFLLRR',
    'KMeans',
    'KMeansProj',
    'MeanShift',
    'OPTICS',
    'Proclus',
    'RecursiveClustering',
    'SingleAgglomerativeClustering',
    'SpectralClustering',
    'SpectralSubspaceRandomization',
    'WardAgglomerativeClustering'
]
columns_names = non_duplicate_columns
should_contain_values = [model_nickname, dataset_id, direction, metric, standardize]
df_missing = get_missing_entries(df_runs_parents, columns_names, should_contain_values)

In [20]:
df = df_runs_raw_parents.copy()
df = df.set_index(['model_nickname', 'dataset_id', 'direction', 'hpo_metric', 'standardize'])[['raised_exception', 'EXCEPTION']]
df_missing = df_missing.join(df, on=non_duplicate_columns)

In [21]:
df_missing

,model_nickname,dataset_id,direction,hpo_metric,standardize,raised_exception,EXCEPTION
0,Clique,46778,maximize,adjusted_rand,True,NaN,NaN
1,Clique,46780,maximize,adjusted_rand,True,NaN,NaN
2,Clique,46781,maximize,adjusted_rand,True,NaN,NaN
3,Clique,46782,maximize,adjusted_rand,True,NaN,NaN
4,Clique,46783,maximize,adjusted_rand,True,NaN,NaN
5,IRFLLRR,46783,maximize,adjusted_rand,True,True,"Every trial failed, no best model was found"
5,IRFLLRR,46783,maximize,adjusted_rand,True,NaN,NaN
5,IRFLLRR,46783,maximize,adjusted_rand,True,NaN,NaN
5,IRFLLRR,46783,maximize,adjusted_rand,True,NaN,NaN
6,SpectralClustering,46782,maximize,adjusted_rand,True,NaN,NaN


In [22]:
df_missing['model_nickname'].unique().tolist()

['Clique', 'IRFLLRR', 'SpectralClustering']

# Plots

In [23]:
all_model_parameters

['damping',
 'sc_n_clusters',
 'lambda_',
 'sampling_ratio',
 'alpha',
 'eps',
 'avg_dims',
 'kmeans_n_clusters',
 'n_similarities',
 'components_size',
 'min_samples',
 'density_threshold',
 'n_partitions',
 'n_clusters',
 'min_cluster_size',
 'c',
 'repetitions',
 'p']

In [24]:
def get_parameters_string(row):
    parameter_names = {
        'components_size': 'q',
        'repetitions': 'R',
        'kmeans_n_clusters': 'C',
        'n_clusters': 'C',
        'min_cluster_size': 'C_{\\text{min}}',
        'eps': '\epsilon',
        'min_samples': 'n_{\\text{min}}',
        'damping': '\lambda',
        'n_similarities': 'm',
        'sampling_ratio': 'r',
        'sc_n_clusters': 'C',
        'density_threshold': '\\tau',
        'n_partitions': 'P',
        'avg_dims': 'd',
        'lambda_': '\lambda',
        'alpha': '\\alpha',
        'c': 'c',
        'p': 'p',
    }
    first = True
    str = ''
    for p in all_model_parameters:
        if not pd.isna(row[p]):
            if not first:
                str += '; '
            else:
                first = False
            value = float(row[p])
            if value.is_integer():
                value = int(value)
                str += f'${parameter_names[p]}={value}$'
            else:
                str += f'${parameter_names[p]}={value:0.2f}$'
    return str        

In [25]:
def highlight_max(df, column_name, level=0):
    df_column = df[column_name]
    max_values = df_column.groupby(level=level).transform('max')
    is_highlighted = df_column.round(3) == max_values.round(3)
    df_css = df.copy().astype(str)
    df_css.loc[:, :] = ''
    df_css[is_highlighted] = 'font-weight: bold'
    return df_css

In [26]:
def highlight_max_index(series_index, df_column, level=0):
    max_values = df_column.groupby(level=level).transform('max')
    is_highlighted = df_column.round(3) == max_values.round(3)
    series_css = series_index.copy().astype(str)
    series_css[:] = ''
    series_css[is_highlighted.values] = 'font-weight: bold'
    return series_css

In [27]:
def underline_2nd_max(df, column_name, level=0):
    df_column = df[column_name]
    # get the second max value
    second_max_values = df_column.groupby(level=level).transform(lambda x: x.nlargest(2).iloc[-1])
    is_underlined = df_column.round(3) == second_max_values.round(3)
    df_css = df.copy().astype(str)
    df_css.loc[:, :] = ''
    df_css[is_underlined] = 'underline: --latex--rwrap'
    return df_css

In [28]:
def underline_2nd_max_index(series_index, df_column, level=0):
    # get the second max value
    second_max_values = df_column.groupby(level=level).transform(lambda x: x.nlargest(2).iloc[-1])
    is_underlined = df_column.round(3) == second_max_values.round(3)
    series_css = series_index.copy().astype(str)
    series_css.loc[:] = ''
    series_css[is_underlined.values] = 'underline: --latex--rwrap'
    return series_css

## Some Models

The following will provide the latex code for a clean table, we only need to make a little adjustement in the first line to delete the "key" and have only one header. For the longtable environment (full data) we need to add the "\*" at the end of lines we dont want to have a page break. We also should replace the entire begin{table} ... end{table} by begin{longtable} ... end{longtable} in the latex file.


In [30]:
datasets_ids = [40685, 39, 61, 182, 40984, 1478, 1568]
datasets_ids = [str(d) for d in datasets_ids]
models_nicknames = ['RecursiveClustering', 'RecursiveClustering_rbf', 'RecursiveClustering_aproximated', 'KMeans', 'HDBSCAN', 'DBSCAN', 'AffinityPropagation', 'OPTICS', 'SpectralSubspaceRandomization', 'WardAgglomerativeClustering']
df = df_runs_parents.copy()
# get time of the child run
df_child = df_runs_raw.copy()
df_child = df_child[['parent_run_id', 'fit_model_return_elapsed_time']]
df_child = df_child.rename(columns={'fit_model_return_elapsed_time': 'Time (s)'})
df = df.join(df_child.set_index('parent_run_id'), on='run_uuid', rsuffix='_child')
df = df.loc[df['dataset_id'].isin(datasets_ids)]
df = df.loc[df['model_nickname'].isin(models_nicknames)]
df = df.loc[df['hpo_metric'] == 'adjusted_rand']
df = df.loc[df['standardize'] == 'True']
df['Parameters'] = df.apply(get_parameters_string, axis=1)
df = df[['model_nickname', 'dataset_name', 'best_adjusted_rand', 'Parameters', 'Time (s)']]
df = df.loc[~((df['model_nickname'] == 'RecursiveClustering_aproximated') & (df['dataset_name'].isin(['iris', 'ecoli'])))]
models_names = {
    'RecursiveClustering_closest_overall_None_False': 'CoHiRF',
    'RecursiveClustering_rbf': 'CoHiRF-RBF',
    'RecursiveClustering_aproximated': 'CoHiRF-1000',
    'KMeans': 'K-Means',
    'HDBSCAN': 'HDBSCAN',
    'DBSCAN': 'DBSCAN',
    'AffinityPropagation': 'Affinity Propagation',
    'OPTICS': 'OPTICS',
    'SpectralSubspaceRandomization': 'SC-SRGF',
    'WardAgglomerativeClustering': "Ward's Method",
}
df = df.replace({'model_nickname': models_names})
df = df.rename(columns={'best_adjusted_rand': 'ARI', 'model_nickname': 'Model', 'dataset_name': 'Dataset'})
df = df.groupby(['Dataset', 'Model']).agg({'ARI': 'mean', 'Time (s)': 'first', 'Parameters': 'first'})

In [31]:
# with time, appendix
df_latex = df.copy()
highlight_max_ari = partial(highlight_max, column_name='ARI')
highlight_max_ari_index = partial(highlight_max_index, df_column=df_latex['ARI'])
underline_2nd_max_ari = partial(underline_2nd_max, column_name='ARI')
underline_2nd_max_ari_index = partial(underline_2nd_max_index, df_column=df_latex['ARI'])
print(df_latex.style.apply(highlight_max_ari, subset='ARI', axis=None).apply_index(highlight_max_ari_index,'index', level=1).apply(underline_2nd_max_ari, subset='ARI', axis=None).apply_index(underline_2nd_max_ari_index, 'index', level=1).format(precision=3, na_rep='No Run', subset='ARI').format(formatter='{:4.3f}',subset='Time (s)').to_latex(hrules=True, clines='skip-last;data', convert_css=True, column_format='llrrl'))

KeyError: "None of [Index([1], dtype='int64')] are in the [columns]"

In [31]:
# no time, main text
df_latex = df.copy()[['ARI', 'Parameters']]
highlight_max_ari = partial(highlight_max, column_name='ARI')
highlight_max_ari_index = partial(highlight_max_index, df_column=df_latex['ARI'])
underline_2nd_max_ari = partial(underline_2nd_max, column_name='ARI')
underline_2nd_max_ari_index = partial(underline_2nd_max_index, df_column=df_latex['ARI'])
print(df_latex.style.apply(highlight_max_ari, subset='ARI', axis=None).apply_index(highlight_max_ari_index,'index', level=1).apply(underline_2nd_max_ari, subset='ARI', axis=None).apply_index(underline_2nd_max_ari_index, 'index', level=1).format(precision=3, na_rep='No Run', subset='ARI').to_latex(hrules=True, clines='skip-last;data', convert_css=True, column_format='p{0.95cm}lp{0.5cm}l'))

\begin{tabular}{p{0.95cm}lp{0.5cm}l}
\toprule
 & key & ARI & Parameters \\
Dataset & Model &  &  \\
\midrule
\multirow[c]{7}{*}{ecoli} & Affinity Propagation & 0.248 & $\lambda=0.58$ \\
 & DBSCAN & 0.345 & $n_{\text{min}}=7$; $\epsilon=0.78$ \\
 & HDBSCAN & 0.398 & $C_{\text{min}}=10$ \\
 & K-Means & 0.719 & $C=6$ \\
 & OPTICS & 0.314 & $n_{\text{min}}=10$ \\
 & \underline{SC-SRGF} & \underline{0.723} & $C=4$; $m=15$; $r=0.80$ \\
 & \bfseries Ward's Method & \bfseries 0.735 & $C=7$ \\
\cline{1-4}
\multirow[c]{7}{*}{har} & Affinity Propagation & 0.313 & $\lambda=1.00$ \\
 & DBSCAN & 0.302 & $n_{\text{min}}=3$; $\epsilon=13.91$ \\
 & HDBSCAN & 0.287 & $C_{\text{min}}=6$ \\
 & K-Means & 0.438 & $C=9$ \\
 & OPTICS & 0.001 & $n_{\text{min}}=4$ \\
 & \bfseries SC-SRGF & \bfseries 0.546 & $C=20$; $m=21$; $r=0.45$ \\
 & \underline{Ward's Method} & \underline{0.511} & $C=4$ \\
\cline{1-4}
\multirow[c]{7}{*}{iris} & Affinity Propagation & 0.477 & $\lambda=0.98$ \\
 & DBSCAN & 0.558 & $n_{\text{m

## All models

In [29]:
# for some reason the css conversion dont work with longtable, so we slightly modify the css for latex
def highlight_max(df, column_name, level=0):
    df_column = df[column_name]
    max_values = df_column.groupby(level=level).transform('max')
    is_highlighted = df_column.round(3) == max_values.round(3)
    df_css = df.copy().astype(str)
    df_css.loc[:, :] = ''
    df_css[is_highlighted] = 'bfseries: '
    return df_css


def highlight_max_index(series_index, df_column, level=0):
    max_values = df_column.groupby(level=level).transform('max')
    is_highlighted = df_column.round(3) == max_values.round(3)
    series_css = series_index.copy().astype(str)
    series_css[:] = ''
    series_css[is_highlighted.values] = 'bfseries: '
    return series_css


def underline_2nd_max(df, column_name, level=0):
    df_column = df[column_name]
    # get the second max value
    second_max_values = df_column.groupby(level=level).transform(lambda x: x.nlargest(2).iloc[-1])
    is_underlined = df_column.round(3) == second_max_values.round(3)
    df_css = df.copy().astype(str)
    df_css.loc[:, :] = ''
    df_css[is_underlined] = 'underline: --rwrap'
    return df_css


def underline_2nd_max_index(series_index, df_column, level=0):
    # get the second max value
    second_max_values = df_column.groupby(level=level).transform(lambda x: x.nlargest(2).iloc[-1])
    is_underlined = df_column.round(3) == second_max_values.round(3)
    series_css = series_index.copy().astype(str)
    series_css.loc[:] = ''
    series_css[is_underlined.values] = 'underline: --rwrap'
    return series_css

In [30]:
model_nickname

['AffinityPropagation',
 'AverageAgglomerativeClustering',
 'Clique',
 'CompleteAgglomerativeClustering',
 'DBSCAN',
 'HDBSCAN',
 'IRFLLRR',
 'KMeans',
 'KMeansProj',
 'MeanShift',
 'OPTICS',
 'Proclus',
 'RecursiveClustering',
 'SingleAgglomerativeClustering',
 'SpectralClustering',
 'SpectralSubspaceRandomization',
 'WardAgglomerativeClustering']

In [31]:
# datasets_ids = [40685, 39, 61, 182, 40984, 1478, 1568]
# datasets_ids = [46773,46774,46775,46776,46777,46778,46779,46780,46781,46782,46783,]
datasets_ids = [46773, 46779, 46783]
datasets_ids = [str(d) for d in datasets_ids]
df = df_runs_parents.copy()
# get time of the child run
df_child = df_runs_raw.copy()
df_child = df_child[['parent_run_id', 'fit_model_return_elapsed_time']]
df_child = df_child.rename(columns={'fit_model_return_elapsed_time': 'Time (s)'})
df = df.join(df_child, on='best_child_run_id', rsuffix='_child')
df = df.loc[df['dataset_id'].isin(datasets_ids)]
df = df.loc[df['hpo_metric'] == 'adjusted_rand']
df = df.loc[df['standardize'] == 'True']
df['Parameters'] = df.apply(get_parameters_string, axis=1)
df = df[['model_nickname', 'dataset_name', 'best_adjusted_rand', 'Parameters', 'Time (s)', 'best_normalized_mutual_info', 'best_silhouette', 'best_davies_bouldin_score']]
df = df.loc[~((df['model_nickname'] == 'RecursiveClustering_aproximated') & (df['dataset_name'].isin(['iris', 'ecoli'])))]
models_names = {
    'RecursiveClustering': 'CoHiRF',
    # 'RecursiveClustering_rbf_None_False': 'CoHiRF-RBF(old)',
    # 'RecursiveClustering_rbf_None_1': 'CoHiRF-RBF',
    # 'RecursiveClustering_closest_overall_1000_False': 'CoHiRF-1000',
    'KMeans': 'K-Means',
    'HDBSCAN': 'HDBSCAN',
    'DBSCAN': 'DBSCAN',
    'AffinityPropagation': 'Affinity Propagation',
    'OPTICS': 'OPTICS',
    'SpectralSubspaceRandomization': 'SC-SRGF',
    'WardAgglomerativeClustering': "Ward's Method",
    'AverageAgglomerativeClustering': 'Average Agglomerative',
    'CompleteAgglomerativeClustering': 'Complete Agglomerative',
    'SingleAgglomerativeClustering': 'Single Agglomerative',
    'IRFLLRR': 'IRFLLRR',
    'Proclus': 'Proclus',
    'Clique': 'Clique',
    'KMeansProj': 'Projective K-Means',
}
df = df.replace({'model_nickname': models_names})
df = df.rename(columns={'best_adjusted_rand': 'ARI', 'model_nickname': 'Model', 'dataset_name': 'Dataset', 'best_normalized_mutual_info': 'NMI', 'best_silhouette': 'Silhouette', 'best_davies_bouldin_score': 'DBI'})
df.to_csv('df_openml_tgcc.csv')
df = df.groupby(['Dataset', 'Model']).agg({'ARI': 'mean', 'Time (s)': 'first', 'Parameters': 'first'})

In [43]:
df

key                                           ARI    Time (s)  \
Dataset          Model                                          
alizadeh-2000-v2 Affinity Propagation    0.362816    0.011788   
                 Average Agglomerative   0.809591    0.019119   
                 Clique                  0.000000    0.130828   
                 CoHiRF                  0.830673    0.016346   
                 Complete Agglomerative  0.539059    0.009286   
                 DBSCAN                  0.250189    0.009145   
                 HDBSCAN                 0.165928    0.025262   
                 IRFLLRR                 0.079580    2.837931   
                 K-Means                 0.814706    0.010609   
                 MeanShift              -0.042740    2.352131   
                 OPTICS                  0.394998    0.049546   
                 Proclus                 0.446010    0.033395   
                 Projective K-Means      0.117385    0.235270   
                 SC-SRGF                 0.599132    0.141913   
                 Single Agglomerative   -0.021908    0.011437   
                 SpectralClustering      0.088318    0.048353   
                 Ward's Method           0.947128    0.011248   
coil-20          Affinity Propagation    0.407319    3.614988   
                 Average Agglomerative   0.080605    0.364514   
                 CoHiRF                  0.464201    0.513552   
                 Complete Agglomerative  0.177666    0.355324   
                 DBSCAN                  0.491158    0.054560   
                 HDBSCAN                 0.677216    2.172935   
                 K-Means                 0.655482    0.072838   
                 MeanShift               0.000077   30.971588   
                 OPTICS                  0.092457    5.224758   
                 Proclus                 0.166905    3.331996   
                 Projective K-Means      0.371780  553.074881   
                 SC-SRGF                 0.775928   22.408264   
                 Single Agglomerative    0.000154    0.981307   
                 Ward's Method           0.719345    0.353966   
garber-2001      Affinity Propagation    0.075380    0.021625   
                 Average Agglomerative   0.146050    0.019162   
                 Clique                  0.061669    0.255860   
                 CoHiRF                  0.298085    0.128058   
                 Complete Agglomerative  0.283306    0.018732   
                 DBSCAN                  0.043459    0.019955   
                 HDBSCAN                 0.026032    0.050493   
                 IRFLLRR                -0.015036    6.606880   
                 K-Means                 0.188167    0.021952   
                 MeanShift               0.016404    2.838703   
                 OPTICS                  0.067677    0.119647   
                 Proclus                 0.046262    0.152017   
                 Projective K-Means      0.114652   21.086727   
                 SC-SRGF                 0.175286    0.162107   
                 Single Agglomerative    0.144825    0.024719   
                 SpectralClustering      0.092858    0.052024   
                 Ward's Method           0.199979    0.020972   

key                                                                             Parameters  
Dataset          Model                                                                      
alizadeh-2000-v2 Affinity Propagation                                       $\lambda=0.86$  
                 Average Agglomerative                                               $C=4$  
                 Clique                                               $P=100$; $\tau=0.50$  
                 CoHiRF                                               $R=3$; $C=2$; $q=30$  
                 Complete Agglomerative                                              $C=6$  
                 DBSCAN                               $\epsilon=39.56$; $n_{\text{min}}=2$  
     

In [36]:
# with time, appendix
df_latex = df.copy()
highlight_max_ari = partial(highlight_max, column_name='ARI')
highlight_max_ari_index = partial(highlight_max_index, df_column=df_latex['ARI'])
underline_2nd_max_ari = partial(underline_2nd_max, column_name='ARI')
underline_2nd_max_ari_index = partial(underline_2nd_max_index, df_column=df_latex['ARI'])
print(df_latex.style.apply(highlight_max_ari, subset='ARI', axis=None).apply_index(highlight_max_ari_index,'index', level=1).apply(underline_2nd_max_ari, subset='ARI', axis=None).apply_index(underline_2nd_max_ari_index, 'index', level=1).format(precision=3, na_rep='No Run', subset='ARI').format(formatter='{:4.3f}',subset='Time (s)').to_latex(hrules=True, clines='skip-last;data', convert_css=False, column_format='llrrl', environment='longtable'))

\begin{longtable}{llrrl}
\toprule
 & key & ARI & Time (s) & Parameters \\
Dataset & Model &  &  &  \\
\midrule
\endfirsthead
\toprule
 & key & ARI & Time (s) & Parameters \\
Dataset & Model &  &  &  \\
\midrule
\endhead
\midrule
\multicolumn{5}{r}{Continued on next page} \\
\midrule
\endfoot
\bottomrule
\endlastfoot
\multirow[c]{17}{*}{alizadeh-2000-v2} & Affinity Propagation & 0.363 & 0.012 & $\lambda=0.86$ \\
 & Average Agglomerative & 0.810 & 0.019 & $C=4$ \\
 & Clique & 0.000 & 0.131 & $P=100$; $\tau=0.50$ \\
 & Complete Agglomerative & 0.539 & 0.009 & $C=6$ \\
 & DBSCAN & 0.250 & 0.009 & $\epsilon=39.56$; $n_{\text{min}}=2$ \\
 & HDBSCAN & 0.166 & 0.025 & $C_{\text{min}}=2$ \\
 & IRFLLRR & 0.080 & 2.838 & $C=8$; $p=0.95$; $c=0.10$; $\lambda=1$; $\alpha=4$ \\
 & K-Means & 0.815 & 0.011 & $C=2$ \\
 & MeanShift & -0.043 & 2.352 &  \\
 & OPTICS & 0.395 & 0.050 & $n_{\text{min}}=2$ \\
 & Proclus & 0.446 & 0.033 & $d=21$; $C=2$ \\
 & Projective K-Means & 0.117 & 0.235 & $C=2$ \\
 & \und

# Debug and explore

In [106]:
df = df_runs_raw_parents.copy()

In [107]:
df

key,status,start_time,end_time,dataset_id,dataset_name,direction,hpo_metric,model_nickname,n_classes,n_features,...,max_memory_used,mutual_info,n_clusters_,normalized_mutual_info,rand_score,silhouette,EXCEPTION,best_child_run_id,parent_run_id,raised_exception
run_uuid,,,,,,,,,,,,,,,,,,,,,
0018714386f44f95ab7141d480ce09f0,FINISHED,1737557036103,1.737557e+12,1044,eye_movements,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,422.740,0.003246,6.0,0.005303,0.358073,0.527841,NaN,756cc869acc94fed89376ae563e075a2,NaN,False
00502ff54da4466191d5fadd2ab0f945,FINISHED,1737556950053,1.737557e+12,7,audiology,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,361.448,0.078387,3.0,0.058583,0.213176,-0.124492,NaN,3d80243180a847ceb7542ac29e31edd8,NaN,False
00b6f8e09ce1419e84214bb9734a9bff,FINISHED,1737589285408,1.737590e+12,23380,cjs,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,491.640,0.271687,51.0,0.125215,0.739143,0.441531,NaN,f7b70e6394ce4793bd5d1924df8d7cf0,NaN,False
00b7b74b49c2418e8cd9108326d76f54,FINISHED,1737504019181,1.737512e+12,46335,primary-tumor_clean,maximize,adjusted_rand,RecursiveClustering,NaN,NaN,...,541.000,2.169609,216.0,0.563600,0.891047,0.359055,NaN,f2af19c022474cc587b3a26c1c1518b7,NaN,False
0200fd7653474a6ab77e2fa4085e294f,FINISHED,1737617015722,1.737619e+12,16,mfeat-karhunen,maximize,adjusted_rand,OPTICS,NaN,NaN,...,1005.424,0.028445,4.0,0.023843,0.120347,-0.058768,NaN,e1c3ab245a3e47d0813cd19cb56802d0,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fb946510558746d29d122b87ee9869da,FINISHED,1737617084498,1.737620e+12,30,page-blocks,maximize,adjusted_rand,OPTICS,NaN,NaN,...,1005.424,0.081613,80.0,0.099852,0.634365,-0.584304,NaN,6578c7710d3d445eb88dd432d6c88b91,NaN,False
fd61b8a547c54e11b898653ebc8769db,FINISHED,1737557019786,1.737557e+12,40984,segment,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,410.420,1.356621,33.0,0.598839,0.856482,0.196516,NaN,d81f797b53d142e68e400b8143cb2586,NaN,False
fdc6bb9a534743c7be6ce026fd67252a,FINISHED,1737556999587,1.737557e+12,61,iris,maximize,adjusted_rand,DBSCAN,NaN,NaN,...,7826.080,0.638874,3.0,0.685822,0.772975,0.507009,NaN,abfd2a33adaf48689df0ef0a86e211b6,NaN,False


In [108]:
df = df.loc[df['status'] == 'RUNNING']

In [109]:
df

key,status,start_time,end_time,dataset_id,dataset_name,direction,hpo_metric,model_nickname,n_classes,n_features,...,max_memory_used,mutual_info,n_clusters_,normalized_mutual_info,rand_score,silhouette,EXCEPTION,best_child_run_id,parent_run_id,raised_exception
run_uuid,,,,,,,,,,,,,,,,,,,,,
0289c581d4a74c6998008fd8c6a23a8b,RUNNING,1737633122174,NaN,41168,NaN,maximize,adjusted_rand,WardAgglomerativeClustering,NaN,NaN,...,NaN,0.145176,25.0,0.073103,0.630174,-0.003434,NaN,NaN,NaN,NaN
0417c1059c264cdf8399c22c311b9876,RUNNING,1737514361699,NaN,41168,NaN,maximize,adjusted_rand,WardAgglomerativeClustering,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
043d0cbd906747819040f8979d7f40d1,RUNNING,1737626372810,NaN,40996,NaN,maximize,adjusted_rand,HDBSCAN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
08ca8fb95ff84adcb2b8c6567a29cc34,RUNNING,1737626671949,NaN,41168,NaN,maximize,adjusted_rand,WardAgglomerativeClustering,NaN,NaN,...,NaN,0.145176,25.0,0.073103,0.630174,-0.003434,NaN,NaN,NaN,NaN
16a5d69973d3475eae567d57d9241d3d,RUNNING,1737516197385,NaN,41027,NaN,maximize,adjusted_rand,AffinityPropagation,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1798d892c9b54192ac736b8013d9229e,RUNNING,1737617191351,NaN,1596,NaN,maximize,adjusted_rand,OPTICS,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1e3053ee2da046f8aaf7a0e0c70ba9dc,RUNNING,1737589047902,NaN,184,NaN,maximize,adjusted_rand,DBSCAN,NaN,NaN,...,NaN,0.000000,1.0,0.000000,0.104322,-1.000000,NaN,NaN,NaN,NaN
230e310a3ff640f6865d1bda85845caf,RUNNING,1737504059995,NaN,1501,NaN,maximize,adjusted_rand,RecursiveClustering,NaN,NaN,...,NaN,0.949910,64.0,0.335039,0.880739,-0.074361,NaN,NaN,NaN,NaN
2490402188d94a7ca9f499eee32a52d3,RUNNING,1737590230603,NaN,1501,NaN,maximize,adjusted_rand,RecursiveClustering,NaN,NaN,...,NaN,0.272477,7.0,0.140823,0.686752,-0.005928,NaN,NaN,NaN,NaN


In [110]:
parent_run_ids = list(df.index)

In [111]:
parent_run_ids

['0289c581d4a74c6998008fd8c6a23a8b',
 '0417c1059c264cdf8399c22c311b9876',
 '043d0cbd906747819040f8979d7f40d1',
 '08ca8fb95ff84adcb2b8c6567a29cc34',
 '16a5d69973d3475eae567d57d9241d3d',
 '1798d892c9b54192ac736b8013d9229e',
 '1e3053ee2da046f8aaf7a0e0c70ba9dc',
 '230e310a3ff640f6865d1bda85845caf',
 '2490402188d94a7ca9f499eee32a52d3',
 '2545cfaa4ce44d4ba25a4c57945f8f4a',
 '26ab3cc84c4a420b8857b1a95ed6f767',
 '2ec4852314ed431f96d3841991fdc743',
 '338ba54bce1345a2a924b657bd5993bf',
 '3692c78f34fd4c3c885c21c7a9071eaa',
 '3eb48652d7dd4c87ac58ddd07b3c5175',
 '4ea354c749a844fea7fa84bbb49cb447',
 '54433df700ed4a78801c8425094c326f',
 '571eb50796254ba09de64dc30acdebc1',
 '6a48f613362447c78e3359b5c9dcc2f3',
 '6cc3d2d840764cbea8c488d14e14b443',
 '6e847cd448bd4b8184b831772de744bd',
 '71d0891f6ac041e38d423fe3328f130e',
 '7239cc6680ec46a880e51cd9e02c3b62',
 '72528d7e02ad4ba1b51ba2ca3335ae83',
 '72cbb804b0d44f4e83b8d6e723bde1d2',
 '7354437c98ce4de9af3822771725e288',
 '7e33df0102fb40968e05fb8ce3e3c909',
 

In [112]:
df = df_runs_raw.copy()
df = df.loc[df['parent_run_id'].isin(parent_run_ids)]

In [113]:
child_run_ids = list(df.index)

In [114]:
runs_to_delete = parent_run_ids + child_run_ids

In [115]:
runs_to_delete

['0289c581d4a74c6998008fd8c6a23a8b',
 '0417c1059c264cdf8399c22c311b9876',
 '043d0cbd906747819040f8979d7f40d1',
 '08ca8fb95ff84adcb2b8c6567a29cc34',
 '16a5d69973d3475eae567d57d9241d3d',
 '1798d892c9b54192ac736b8013d9229e',
 '1e3053ee2da046f8aaf7a0e0c70ba9dc',
 '230e310a3ff640f6865d1bda85845caf',
 '2490402188d94a7ca9f499eee32a52d3',
 '2545cfaa4ce44d4ba25a4c57945f8f4a',
 '26ab3cc84c4a420b8857b1a95ed6f767',
 '2ec4852314ed431f96d3841991fdc743',
 '338ba54bce1345a2a924b657bd5993bf',
 '3692c78f34fd4c3c885c21c7a9071eaa',
 '3eb48652d7dd4c87ac58ddd07b3c5175',
 '4ea354c749a844fea7fa84bbb49cb447',
 '54433df700ed4a78801c8425094c326f',
 '571eb50796254ba09de64dc30acdebc1',
 '6a48f613362447c78e3359b5c9dcc2f3',
 '6cc3d2d840764cbea8c488d14e14b443',
 '6e847cd448bd4b8184b831772de744bd',
 '71d0891f6ac041e38d423fe3328f130e',
 '7239cc6680ec46a880e51cd9e02c3b62',
 '72528d7e02ad4ba1b51ba2ca3335ae83',
 '72cbb804b0d44f4e83b8d6e723bde1d2',
 '7354437c98ce4de9af3822771725e288',
 '7e33df0102fb40968e05fb8ce3e3c909',
 

In [116]:
run_uuid_query = [f"'{run_id}'" for run_id in runs_to_delete]
run_uuid_query = ', '.join(run_uuid_query)

In [117]:
query = f"""
UPDATE runs
SET lifecycle_stage = 'deleted'
WHERE run_uuid IN ({run_uuid_query}) 
"""
with engine.begin() as conn:
    conn.execute(text(query))

for i, row in df_runs_raw.iterrows():
    run_id = row.run_id
    model_name = row['params.model_name']
    with mlflow.start_run(run_id) as run:
        mlflow.log_param('model_nickname', model_name)    